In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ModuleNotFoundError as e:
    print("not in colab")
    pass
import os
base_dir = "/content/drive/MyDrive/semeval2022"
if not os.path.exists(base_dir):
  !pip install -r requirements.txt
  base_dir = ""
else:
  !pip install -r /content/drive/MyDrive/semeval2022/requirements.txt
  !cp -rf /content/drive/MyDrive/semeval2022/*.py . 
  !cp -rf /content/drive/MyDrive/semeval2022/utils .
  !cp -rf /content/drive/MyDrive/semeval2022/model .

In [1]:
from utils.util import parse_args, get_reader, load_model, get_trainer, get_out_filename, write_eval_performance, get_tagset
import time
from torch.utils.data import DataLoader
from utils.util import wnut_iob
import os

In [2]:
checkpoint = os.path.join(base_dir, "roberta-train/lightning_logs/version_0/checkpoints/epoch=4-step=4784.ckpt")
checkpoint = os.path.join(base_dir, "roberta-train/lightning_logs/version_0/checkpoints/roberta-train_timestamp_1638429490.282762_final.ckpt")
#checkpoint = "./roberta-finetune/lightning_logs/version_0/checkpoints/"
checkpoint = os.path.join(base_dir, "roberta-finetune/lightning_logs/version_1/checkpoints/finetune_timestamp_1638516609.385282_final.ckpt")
train_file = os.path.join(base_dir, "./training_data/EN-English/en_train.conll")
dev_file = os.path.join(base_dir, "./training_data/EN-English/en_dev.conll")
output_dir = os.path.join(base_dir, "roberta-evaluate")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
iob_tagging = wnut_iob

In [3]:
test_data = get_reader(file_path=dev_file, target_vocab=wnut_iob, max_instances=-1, max_length=50, encoder_model="roberta-base")
model = load_model(checkpoint, tag_to_id=wnut_iob)
trainer = get_trainer(is_test=True)
#out = trainer.test(model, test_dataloaders=DataLoader(test_data, batch_size=8, collate_fn=model.collate_batch))
eval_file, eval_detail_file = get_out_filename(output_dir, checkpoint, prefix="eval")
#write_eval_performance(out, eval_file)

2021-12-06 15:32:48 - INFO - reader - Reading file ./training_data/EN-English/en_dev.conll
2021-12-06 15:32:49 - INFO - reader - Finished reading 800 instances from file ./training_data/EN-English/en_dev.conll
Some weights of the model checkpoint at ./roberta-retrained/checkpoint-5000 were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./roberta-r

In [21]:
batch_size = 8
test_dataloader = DataLoader(test_data, batch_size=batch_size, collate_fn=model.collate_batch)
sentences = test_data.sentences
ner_tags = test_data.ner_tags
pos_to_singel_word_map = test_data.pos_to_single_word_maps
f = open(eval_detail_file, "w")
f.write("token\tlabel\tpred\n")

for idx, batch in enumerate(test_dataloader):
    output = model.perform_forward_step(batch)
    pred_result = output["pred_results"]
    raw_pred_results = output["raw_pred_results"]
    for i in range(batch_size):
        sentence = sentences[idx*batch_size+i]
        pos_to_singel_word = pos_to_singel_word_map[idx*batch_size+i]
        ner_tag = ner_tags[idx*batch_size+i]
        input_ids = batch[0][i]
        pred_token_tag = pred_result[i]
        raw_pred_token_tag = raw_pred_results[i]
        metadata_token_tag = batch[3][i]
        meta_labels = []
        pred_labels = []
        sentence_subtokens = []
        for (start_pos, end_pos), (pred_start_pos, pred_end_pos) in zip(metadata_token_tag, pred_token_tag):
            sub_tokens = test_data.tokenizer.convert_ids_to_tokens(input_ids[start_pos: end_pos+1])
            sentence_subtokens.extend(sub_tokens)
            pred_sub_tokens = test_data.tokenizer.convert_ids_to_tokens(input_ids[start_pos: end_pos+1])
            tag = metadata_token_tag[(start_pos, end_pos)]
            pred_tag = pred_token_tag[(pred_start_pos, pred_end_pos)]
            for sub_token1, sub_token2 in zip(sub_tokens, pred_sub_tokens):
                meta_labels.append(tag)
                pred_labels.append(pred_tag)
                #f.write("{}{}{}{}{}{}{}".format(sub_token1, ",", tag, ",", sub_token2, ",", pred_tag))
                #f.write("\n")
        for (start, end) in pos_to_singel_word:
            single_word_tokens = sentence_subtokens[start:end]
            word = "".join(single_word_tokens)
            word_meta_tag = ner_tag[start]
            word_pred_tag = raw_pred_token_tag[start]
            f.write("{}\t{}\t{}".format(word, word_meta_tag, word_pred_tag))
            f.write("\n")
            
f.close() 

In [24]:
import pandas as pd
result_data = pd.read_csv(eval_detail_file, delimiter="\t")

In [26]:
error_data = result_data[result_data["label"] != result_data["pred"]]
print(error_data)

            token   label    pred
15            the    B-CW       O
16            wiz    I-CW       O
76         marble       O   B-LOC
77         canyon       O   I-LOC
79          cliff       O   B-LOC
...           ...     ...     ...
13411      kyselo       O  B-PROD
13413   krkonoÅ¡e       O    B-CW
13419      boiled  B-PROD       O
13420         egg  I-PROD       O
13437  vaudeville    B-CW  B-CORP

[630 rows x 3 columns]


In [37]:
test_data.tokenizer("among others , he was chairman of the standing conference of regional sports federations and member of the südwestrundfunk broadcasting council")

{'input_ids': [0, 31636, 643, 2156, 37, 21, 2243, 9, 5, 2934, 1019, 9, 2174, 1612, 32584, 1635, 8, 919, 9, 5, 579, 2768, 417, 10823, 2070, 1187, 506, 6435, 16687, 1676, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [35]:
test_data.sentences[0].split(" ")

['',
 'ease',
 'on',
 'down',
 'the',
 'road',
 '—',
 'charlie',
 'smalls',
 '(',
 'diana',
 'ross',
 'and',
 'michael',
 'jackson',
 'in',
 'the',
 'wiz',
 ')']